In [ ]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine


pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# create engine
SQLITE_DB = 'sqlite/umls_py.db'
engine = create_engine(f"sqlite:///{SQLITE_DB}")

### Concept Nodes - Concept Unique Identifier (CUI)

In [ ]:
def cui_nodes():
    sql = """
    SELECT DISTINCT MRCONSO.CUI, MRCONSO.STR, (STY_GROUP || ';' || 'Concept')
    FROM MRCONSO, MRSTY, SRGRP
    WHERE MRCONSO.CUI = MRSTY.CUI
    AND MRSTY.STY = SRGRP.STY
    AND MRCONSO.SAB IN ('{0}', '{1}', '{2}', '{3}', '{4}', '{5}', '{6}', '{7}', '{8}')
    AND MRCONSO.SUPPRESS = 'N'
    AND MRCONSO.ISPREF = 'Y'
    AND MRCONSO.TS = 'P'
    AND MRCONSO.STT = 'PF'
    AND MRCONSO.TTY NOT IN ('{9}', '{10}')
    AND MRCONSO.TTY NOT LIKE 'MTH%';
    """.format('ATC', 'GO', 'HGNC', 'HPO', 'MED-RT', 'NCI', 'OMIM', 'RXNORM', 'SNOMEDCT_US', 'PTGB', 'SYGB')
    df = pd.read_sql_query(sql, engine).drop_duplicates(
        ['CUI']).replace(np.nan, "")
    df.columns = ['Cui:ID', 'prefLabel', ':LABEL']
    df.to_csv(
        path_or_buf='../../../import/cuiNode.csv',
        index=False,
        header=True
    )
    return df


cuiNode = cui_nodes()
print({len(cuiNode)})
cuiNode.head()

In [ ]:
# Create a set of all umls_cuis contained in cuiNode.csv
cui_set = set(cuiNode['Cui:ID'])

### Atom/Code Nodes - Atomic Unique Identifier (AUI)

In [ ]:
def aui_nodes():
    sql = """
    SELECT DISTINCT MRCONSO.AUI,
                    MRCONSO.STR,
                    MRCONSO.SAB, 
                    MRCONSO.CODE, 
                    MRCONSO.TTY, 
                    MRCONSO.ISPREF, 
                    MRCONSO.TS, 
                    MRCONSO.STT, 
                    (MRCONSO.SAB || ';' || 'Atom')
    FROM MRCONSO
    WHERE MRCONSO.SAB IN ('{0}', '{1}', '{2}', '{3}', '{4}', '{5}', '{6}', '{7}', '{8}')
    AND MRCONSO.SUPPRESS = 'N'
    AND MRCONSO.TTY NOT IN ('{9}', '{10}')
    AND MRCONSO.TTY NOT LIKE 'MTH%';
    """.format('ATC', 'GO', 'HGNC', 'HPO', 'MED-RT', 'NCI', 'OMIM', 'RXNORM', 'SNOMEDCT_US', 'PTGB', 'SYGB')
    df = pd.read_sql_query(sql, engine).drop_duplicates().replace(np.nan, "")
    df.columns = ['Atom:ID', 'str', 'sab', 'code', 'tty',
                  'ispref', 'ts', 'stt', ':LABEL']
    df.to_csv(
        path_or_buf='../../../import/atomNode.csv',
        index=False,
        header=True
    )
    return df


atomNode = aui_nodes()
print({len(atomNode)})
atomNode.head()

In [ ]:
# Create a set of all umls_auis (atoms) contained in atomNode.csv
aui_set = set(atomNode['Atom:ID'])

### Semantic Type Nodes - Semantic Unique Identifier (TUI)

In [ ]:
def sty_nodes():
    sql = f"""
    SELECT DISTINCT MRSTY.TUI, 
                    MRSTY.STY, 
                    MRSTY.STN, 
                    SRGRP.STY_GROUP_ABBREV, 
                    'SemanticType' AS ":LABEL"
    FROM MRSTY, 
         SRGRP 
    WHERE MRSTY.STY = SRGRP.STY 
    AND MRSTY.CUI IN {tuple(cui_set)};
    """
    df = pd.read_sql_query(sql, engine).drop_duplicates().replace(np.nan, "")
    df.columns = ['Tui:ID', 'sty', 'stn', 'sty_group', ':LABEL']
    df.to_csv(
        path_or_buf="../../../import/styNode.csv",
        index=False,
        header=True
    )
    return df


styNode = sty_nodes()
print({len(styNode)})
styNode.head()

### Concept -> Semantic Type Relationship

In [ ]:
def has_sty_rel():
    sql = f"""
    SELECT DISTINCT MRCONSO.CUI, 
                    MRSTY.TUI, 
                    'HAS_STY' AS ":TYPE"
    FROM MRCONSO
    JOIN MRSTY ON MRCONSO.CUI = MRSTY.CUI
    WHERE MRCONSO.CUI IN {tuple(cui_set)}
    AND MRCONSO.SUPPRESS = 'N';
    """
    df = pd.read_sql_query(sql, engine).drop_duplicates().replace(np.nan, "")
    df.columns = [":START_ID", ":END_ID", ":TYPE"]
    df.to_csv(
        path_or_buf="../../../import/cui_sty_rel.csv", header=True, index=False
    )

    return df


has_sty = has_sty_rel()
print({len(has_sty)})
has_sty.head()

### Atom -> Concept Relationship

In [ ]:
def has_cui_rel():
    sql = f"""
    SELECT DISTINCT AUI, CUI, 'HAS_CUI'
    FROM MRCONSO
    WHERE AUI IN {tuple(aui_set)}
    AND CUI IN {tuple(cui_set)}
    AND SUPPRESS = 'N';
    """
    df = pd.read_sql_query(sql, engine).drop_duplicates().replace(np.nan, "")
    df.columns = [":START_ID", ":END_ID", ":TYPE"]
    df.to_csv(
        path_or_buf="../../../import/has_cui.csv", header=True, index=False
    )
    return df


has_atom = has_aui_rel()
print({len(has_atom)})
has_atom.head()

### Concept -> Concept Relationships 
##### Note: There are many of these - further filtering advised

- Should also remove redundant reverse relationships that provide no value?

In [ ]:
def cui_cui_rel():
    sql = f"""
    SELECT CUI2, CUI1, CASE WHEN RELA = '' THEN REL ELSE RELA END AS ":TYPE"
    FROM MRREL
    WHERE MRREL.CUI2 IN {tuple(cui_set)}
    AND MRREL.CUI1 IN {tuple(cui_set)}
    AND MRREL.SUPPRESS = 'N';
    """
    df = pd.read_sql_query(sql, engine).drop_duplicates().replace(np.nan, "")
    df.columns = [":START_ID", ":END_ID", ":TYPE"]
    df = (
        df[
            (df[":START_ID"] != df[":END_ID"])
            & (df[":TYPE"] != "SIB")
            & (df[":TYPE"] != "SY")
        ]
        .drop_duplicates()
        .replace(np.nan, "")
    )
    df[":TYPE"] = df[":TYPE"].str.upper()
    df[":TYPE"] = df[":TYPE"].str.replace("-", "_")
    df.to_csv("../../../import/cui_rel.csv", header=True, index=False)
    return df


cui_rel = cui_cui_rel()
print({len(cui_rel)})
cui_rel.head()

In [ ]:
# # Relationships distilled to a single direction (no redundant relationships)
# cuirel = cui_rel[cui_rel[':TYPE'].isin(['INVERSE_ISA', 'SUBSET_INCLUDES_CONCEPT', 'INACTIVE_INGREDIENT_OF', 'INGREDIENT_OF', 'HAS_DOSE_FORM'
#                                         'HAS_METHOD', 'TRADENAME_OF', 'CONSISTS_OF', 'DISEASE_HAS_ABNORMAL_CELL', 'MANIFESTATION_OF'
#                                         'FINDING_SITE_OF', 'ACTIVE_INGREDIENT_OF', 'ASSOCIATED_MORPHOLOGY_OF', 'IS_FINDING_OF_DISEASE', 'HAS_ALLELIC_VARIANT'
#                                         'PAR', 'INTERPRETS', 'PROCESS_INVOLVES_GENE', 'IS_NOT_FINDING_OF_DISEASE', 'ACTIVE_MOIETY_OF'
#                                         'LATERALITY_OF', 'RB', 'HAS_PARENT', 'PART_OF', 'IS_NOT_ABNORMAL_CELL_OF_DISEASE'
#                                         'PHENOTYPE_OF', 'DOSEFORMGROUP_OF', 'PATHOLOGICAL_PROCESS_OF', 'HAS_OCCURRENCE', 'PATHWAY_HAS_GENE_ELEMENT'
#                                         'HAS_ENTIRE_ANATOMY_STRUCTURE', 'INTERPRETATION_OF', 'HAS_COMPONENT', 'CAUSE_OF', 'HAS_DIRECT_PROCEDURE_SITE'
#                                         'HAS_PHYSIOLOGIC_EFFECT', 'CHEMOTHERAPY_REGIMEN_HAS_COMPONENT', 'IS_ASSOCIATED_ANATOMIC_SITE_OF', 'CAUSATIVE_AGENT_OF', 'MAPPED_FROM'
#                                         'DISEASE_HAS_NORMAL_TISSUE_ORIGIN', 'ROLE_PLAYED_BY', 'DISEASE_MAY_HAVE_CYTOGENETIC_ABNORMALITY', 'DISEASE_MAY_HAVE_FINDING', 'HAS_PRECISE_ACTIVE_INGREDIENT'
#                                         'BASIS_OF_STRENGTH_SUBSTANCE_OF', 'IS_VALUE_FOR_GDC_PROPERTY', 'DISEASE_MAY_HAVE_MOLECULAR_ABNORMALITY', 'GENE_PRODUCT_ENCODED_BY_GENE', 'MECHANISM_OF_ACTION_OF'
#                                         'IS_PRIMARY_ANATOMIC_SITE_OF_DISEASE', 'PRECISE_INGREDIENT_OF', 'DISEASE_IS_STAGE', 'HAS_DISPOSITION', 'HAS_TEMPORAL_CONTEXT'
#                                         'BIOLOGICAL_PROCESS_INVOLVES_GENE_PRODUCT', 'IS_NORMAL_CELL_ORIGIN_OF_DISEASE', 'SUBJECT_RELATIONSHIP_CONTEXT_OF', 'HAS_INTENT', 'CLINICAL_COURSE_OF'
#                                         'PRESENTATION_STRENGTH_NUMERATOR_UNIT_OF', 'HAS_UNIT_OF_PRESENTATION', 'UNIT_OF_PRESENTATION_OF', 'PRESENTATION_STRENGTH_DENOMINATOR_UNIT_OF', 'HAS_MEMBER'
#                                         'GENE_IN_CHROMOSOMAL_LOCATION', 'HAS_INGREDIENTS', 'HAS_CTCAE_5_PARENT', 'CONTAINS', 'GENE_PRODUCT_IS_ELEMENT_IN_PATHWAY'
#                                         'HAS_INHERITANCE_TYPE', 'REGULATES', 'DIRECT_MORPHOLOGY_OF', 'FORM_OF', 'GENE_INVOLVED_IN_MOLECULAR_ABNORMALITY'
#                                         'EO_DISEASE_HAS_ASSOCIATED_EO_ANATOMY', 'DISEASE_MAPPED_TO_GENE', 'SPECIMEN_OF', 'GENETIC_BIOMARKER_RELATED_TO', 'HAS_DIRECT_SUBSTANCE'
#                                         'NEGATIVELY_REGULATES', 'POSITIVELY_REGULATES', 'USES_DEVICE', 'FOCUS_OF', 'HAS_PROCEDURE_CONTEXT'
#                                         'DIRECT_DEVICE_OF', 'HAS_FINDING_CONTEXT', 'CLASSIFIED_AS', 'INDIRECT_PROCEDURE_SITE_OF', 'HAS_THERAPEUTIC_CLASS'
#                                         'ANATOMIC_STRUCTURE_IS_PHYSICAL_PART_OF', 'CONCENTRATION_STRENGTH_NUMERATOR_UNIT_OF', 'HAS_CONCENTRATION_STRENGTH_DENOMINATOR_UNIT', 'IS_MECHANISM_OF_ACTION_OF_CHEMICAL_OR_DRUG', 'IS_MODIFICATION_OF'
#                                         'HAS_FINDING_METHOD', 'IS_PHYSIOLOGIC_EFFECT_OF_CHEMICAL_OR_DRUG', 'HAS_ASSOCIATED_PROCEDURE', 'GENE_PRODUCT_SEQUENCE_VARIATION_ENCODED_BY_GENE_MUTANT', 'HAS_ACCESS'
#                                         'CONTRAINDICATED_CLASS_OF', 'IS_STRUCTURAL_DOMAIN_OR_MOTIF_OF_GENE_PRODUCT', 'HAS_PROPERTY', 'IS_GRADE_OF_DISEASE', 'HAS_REALIZATION'
#                                         'HAS_FREE_ACID_OR_BASE_FORM', 'IS_NOT_NORMAL_CELL_ORIGIN_OF_DISEASE', 'MOVED_FROM', 'IS_MOLECULAR_ABNORMALITY_OF_DISEASE', 'HAS_PROCEDURE_SITE'
#                                         'ASSOCIATED_FINDING_OF', 'DISEASE_MAY_HAVE_ABNORMAL_CELL', 'HAS_REVISION_STATUS', 'OCCURS_BEFORE', 'GENE_HAS_PHYSICAL_LOCATION'
#                                         'USES_SUBSTANCE', 'SCALE_TYPE_OF', 'HAS_REGIONAL_PART', 'SPECIMEN_PROCEDURE_OF', 'SPECIMEN_SUBSTANCE_OF'
#                                         'CONSIDER', 'REPLACES', 'PRIMARY_MAPPED_FROM', 'HAS_SURGICAL_APPROACH', 'CHARACTERIZED_BY'
#                                         'PATHOGENESIS_OF_DISEASE_INVOLVES_GENE', 'WAS_A', 'TIME_ASPECT_OF', 'DISEASE_EXCLUDES_PRIMARY_ANATOMIC_SITE', 'GENE_PRODUCT_HAS_BIOCHEMICAL_FUNCTION'
#                                         'HAS_PHARMACEUTICAL_STATE_OF_MATTER', 'PHARMACEUTICAL_BASIC_DOSE_FORM_OF', 'REFERRED_TO_BY', 'HAS_PHARMACEUTICAL_ADMINISTRATION_METHOD', 'DISEASE_EXCLUDES_MOLECULAR_ABNORMALITY'
#                                         'IS_TARGET', 'HAS_PHARMACEUTICAL_TRANSFORMATION', 'PHARMACEUTICAL_INTENDED_SITE_OF', 'PHARMACEUTICAL_RELEASE_CHARACTERISTICS_OF', 'HAS_PCDC_AML_PERMISSIBLE_VALUE'
#                                         'HAS_PHARMACEUTICAL_RELEASE_CHARACTERISTICS', 'HAS_LAB_NUMBER', 'HAS_INHERENT_ATTRIBUTE', 'CONTRAINDICATED_MECHANISM_OF_ACTION_OF', 'DISEASE_EXCLUDES_CYTOGENETIC_ABNORMALITY'
#                                         'OTHER_MAPPED_FROM', 'VALUE_SET_IS_PAIRED_WITH', 'HAS_SPECIMEN_SOURCE_TOPOGRAPHY', 'GENE_PRODUCT_HAS_ASSOCIATED_ANATOMY', 'IS_PCDC_EWS_PERMISSIBLE_VALUE_FOR_VARIABLE'
#                                         'HAS_PROCESS_DURATION', 'DISEASE_HAS_ASSOCIATED_GENE', 'HAS_PROCESS_OUTPUT', 'PROCESS_OUTPUT_OF', 'HAS_DOSE_FORM_INTENDED_SITE'
#                                         'CONSTITUTIONAL_PART_OF', 'EO_DISEASE_MAPS_TO_HUMAN_DISEASE', 'HAS_DIRECT_SITE', 'ACCESS_DEVICE_USED_BY', 'IS_PCDC_GCT_PERMISSIBLE_VALUE_FOR_VARIABLE'
#                                         'DOSE_FORM_ADMINISTRATION_METHOD_OF', 'HAS_DOSE_FORM_ADMINISTRATION_METHOD', 'DOSE_FORM_RELEASE_CHARACTERISTIC_OF', 'HAS_CONTRAINDICATED_PHYSIOLOGIC_EFFECT', 'EO_DISEASE_HAS_PROPERTY_OR_ATTRIBUTE'
#                                         'HAS_DOSE_FORM_TRANSFORMATION', 'IS_NOT_NORMAL_TISSUE_ORIGIN_OF_DISEASE', 'IS_CYTOGENETIC_ABNORMALITY_OF_DISEASE', 'ANSWER_TO_IS_STERILE', 'IS_STERILE'
#                                         'HAS_TECHNIQUE', 'IS_SERONET_PERMISSIBLE_VALUE_FOR_VARIABLE', 'HAS_PCDC_ALL_PERMISSIBLE_VALUE', 'IS_ABNORMALITY_OF_GENE', 'FINDING_INFORMER_OF'
#                                         'HAS_FINDING_INFORMER', 'STRUCTURAL_CLASS_OF', 'HAS_PCDC_DATA_TYPE', 'MAY_BE_ASSOCIATED_DISEASE_OF_DISEASE', 'BIOLOGICAL_PROCESS_IS_PART_OF_PROCESS'
#                                         'PRIORITY_OF', 'BRANCH_OF', 'HAS_INC_PARENT', 'GENE_PRODUCT_VARIANT_OF_GENE_PRODUCT', 'HAS_ABSORBABILITY'
#                                         'HAS_ROUTE_OF_ADMINISTRATION', 'HAS_TRIBUTARY', 'DURING', 'ALLELE_HAS_ABNORMALITY', 'HAS_INDIRECT_MORPHOLOGY'
#                                         'HAS_PART_ANATOMY_STRUCTURE', 'GENE_PRODUCT_HAS_CHEMICAL_CLASSIFICATION', 'PROCESS_ALTERED_BY_ALLELE', 'HAS_COMPOSITIONAL_MATERIAL', 'COMPLEX_HAS_PHYSICAL_PART'
#                                         'REGIMEN_HAS_ACCEPTED_USE_FOR_DISEASE', 'PROCEDURE_HAS_TARGET_ANATOMY', 'HAS_BASIC_DOSE_FORM', 'HAS_CTDC_VALUE', 'PROCEDURE_DEVICE_OF'
#                                         'ALLELE_IN_CHROMOSOMAL_LOCATION', 'DISEASE_HAS_ASSOCIATED_DISEASE', 'MAY_BE_PREVENTED_BY', 'PROCEDURE_MORPHOLOGY_OF', 'HAS_PROCEDURE_MORPHOLOGY'
#                                         'MAY_BE_NORMAL_CELL_ORIGIN_OF_DISEASE', 'KIND_IS_RANGE_OF', 'DISEASE_MAY_HAVE_NORMAL_CELL_ORIGIN', 'EXTENDED_TO_BY_PROCESS', 'GENE_PRODUCT_IS_BIOMARKER_TYPE'
#                                         'BIOMARKER_TYPE_INCLUDES_GENE_PRODUCT', 'RECEIVES_DRAINAGE_FROM', 'ALLELE_HAS_ACTIVITY', 'USES_ENERGY', 'KIND_IS_DOMAIN_OF'
#                                         'ROLE_HAS_DOMAIN', 'HAS_ICDC_VALUE', 'BIOLOGICAL_PROCESS_HAS_INITIATOR_PROCESS', 'ENZYME_METABOLIZES_CHEMICAL_OR_DRUG', 'GENE_PRODUCT_AFFECTED_BY_CHEMICAL_OR_DRUG'
#                                         'INVERSE_RELATIVE_TO', 'SPECIMEN_SOURCE_MORPHOLOGY_OF', 'INHERENT_LOCATION_OF', 'DISEASE_HAS_METASTATIC_ANATOMIC_SITE', 'DEVICE_INTENDED_SITE_OF'
#                                         'HAS_INDIRECT_DEVICE', 'GENE_PRODUCT_EXPRESSED_IN_TISSUE', 'SEVERITY_OF', 'CONTINUOUS_PROXIMALLY_WITH', 'CONTINUOUS_DISTALLY_WITH'
#                                         'SITE_OF_METABOLISM', 'ALIAS_OF', 'ARTICULATES_WITH', 'HAS_ALIAS', 'BOUNDS'
#                                         'SUPERIOR_TO', 'PROXIMAL_TO', 'DISTAL_TO', 'HAS_ORIGIN', 'HAS_STATE_OF_MATTER'
#                                         'STATE_OF_MATTER_OF', 'BIOLOGICAL_PROCESS_HAS_RESULT_BIOLOGICAL_PROCESS', 'HAS_PHARMACOKINETICS', 'PHARMACOKINETICS_OF', 'PROCEDURE_HAS_EXCISED_ANATOMY'
#                                         'IS_LOCATION_OF_BIOLOGICAL_PROCESS', 'BIOLOGICAL_PROCESS_HAS_ASSOCIATED_LOCATION', 'PROCEDURE_HAS_COMPLETELY_EXCISED_ANATOMY', 'HAS_ARTERIAL_SUPPLY', 'INSERTION_OF'
#                                         'CHEMICAL_OR_DRUG_AFFECTS_ABNORMAL_CELL', 'ARTERIAL_SUPPLY_OF', 'ABNORMAL_CELL_AFFECTED_BY_CHEMICAL_OR_DRUG', 'HAS_SPECIMEN_SOURCE_IDENTITY', 'UNITS_OF'
#                                         'NERVE_SUPPLY_OF', 'HAS_UNITS', 'CHEMICAL_OR_DRUG_INITIATES_BIOLOGICAL_PROCESS', 'HAS_RECIPIENT_CATEGORY', 'CONTRAINDICATED_WITH_DISEASE'
#                                         'GENE_PRODUCT_MALFUNCTION_ASSOCIATED_WITH_DISEASE', 'RECEIVES_INPUT_FROM', 'RECEIVES_ATTACHMENT_FROM', 'ATTACHES_TO', 'BIOMARKER_TYPE_INCLUDES_GENE'
#                                         'POSTERIOR_TO', 'LATERAL_TO', 'VENOUS_DRAINAGE_OF', 'HAS_VENOUS_DRAINAGE', 'EO_DISEASE_HAS_ASSOCIATED_CELL_TYPE'
#                                         'PARTIALLY_EXCISED_ANATOMY_HAS_PROCEDURE', 'GENE_IS_BIOMARKER_OF', 'HAS_TARGET_POPULATION', 'DISEASE_IS_MARKED_BY_GENE'])].drop_duplicates().replace(np.nan, '')  # value_counts()

### Atom -> Atom Parent/Children Relationships

In [ ]:
def child_of_rel():
    sql = f"""
    SELECT DISTINCT h1.PAUI, h1.AUI, 'CHILD_OF'
    FROM MRHIER h1,
         MRCONSO c1,
         MRCONSO c2
    WHERE h1.AUI = c1.AUI
    AND h1.PAUI = c2.AUI
    AND h1.AUI IN {tuple(aui_set)}
    AND c1.SUPPRESS = 'N'
    AND c2.SUPPRESS = 'N'
    """
    df = pd.read_sql_query(sql, engine)
    df.columns = [":START_ID", ":END_ID", ":TYPE"]
    df = df[df[":START_ID"] != df[":END_ID"]
            ].drop_duplicates().replace(np.nan, "")
    df.to_csv(
        path_or_buf="../../../import/child_of.csv", header=True, index=False
    )
    return df


child_of = child_of_rel()
print({len(child_of)})
child_of.head()

### Semantic Type -> Semantic Type Parent/Children Relationships

In [ ]:
def sty_isa():
    sql = """
    SELECT DISTINCT s2.UI, s3.UI, s.RL
    FROM SRSTR s
    JOIN SRDEF s2 ON s.STY_RL1 = s2.STY_RL
    JOIN SRDEF s3 ON s.STY_RL2 = s3.STY_RL
    WHERE s2.UI != s3.UI
    AND s.RL = 'isa';
    """
    df = pd.read_sql_query(sql, engine).drop_duplicates().replace(np.nan, "")
    df.columns = [":START_ID", ":END_ID", ":TYPE"]
    df = (
        df[df[":START_ID"] != df[":END_ID"]]
        .drop_duplicates()
        .replace(np.nan, "")
    )
    df[":TYPE"] = df[":TYPE"].str.upper()
    df.to_csv(
        path_or_buf="../../../import/sty_sty_rel.csv", header=True, index=False
    )
    return df


styisa = sty_isa()
styisa[':TYPE'] = 'STY_ISA'
print({len(styisa)})
styisa.head()

### neo4j-admin import 